#This code is for scraping rare diseases from orpha.net

In [24]:
import requests
from bs4 import BeautifulSoup
import json 

In [25]:
#basic orpha net url
base_url = "https://www.orpha.net/en/disease/list/"

#append each alphabet of diseases
suffixes = ['0'] + [chr(i) for i in range(ord('a'), ord('z') + 1)]

urls = [f"{base_url}{suffix}" for suffix in suffixes]


In [26]:
#scrape data from website 
data = {"disease": []}

#url = "https://www.orpha.net/en/disease/list/d"
for url in urls:
    try:
        response = requests.get(url)
        response.raise_for_status() 
        soup = BeautifulSoup(response.text, "html.parser")

        #find diseases in html
        results_div = soup.find("div", class_="mx-4 results")
        if results_div:

            disease_list = results_div.find("ul", class_="list-unstyled")
            if disease_list:
                for li in disease_list.find_all("li"):
                    tag_a = li.find("a")
                    if tag_a and "href" in tag_a.attrs:
                        name = tag_a.text.strip()
                        url_disease = tag_a["href"].strip()
                        number = url_disease.split("/")[-1]

                        data["disease"].append({
                            "Name": name, 
                            "Orphanet_num": number,
                            "Orphanet_url": url_disease
                        })

    except Exception as e:
        print(f"Error processing URL {url}: {e}")
    
print(len(data["disease"]))

23124


In [27]:
output_dir = "./disease_json/data.json"
with open(output_dir, "w", encoding="utf-8") as json_file:
    json.dump(data, json_file, indent=4, ensure_ascii=False)